In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path 
import os
import datetime as dt
import requests
import json
from intuitlib.client import AuthClient
import urllib.parse
from time import perf_counter
import re
import yaml

In [2]:
class Job:
    def __init__(self):
        base_dir = Path("c:/Users/ZheRao/OneDrive - Monette Farms/Monette Farms Team Site - Innovation Projects/Production/Database")
        self.base_dir = base_dir
        self.today = dt.date.today()
        month_format = "".join(["0",str(self.today.month)]) if self.today.month < 10 else str(self.today.month)
        self.us_companies = ["MFUSA", "MFAZ", "MSUSA", "MPUSA"]
        self.company_names = self.us_companies + ["MSL", "NexGen", "MFBC", "MPL", "MFL"]
        self.raw_path = {
            "QBO": {
                "Raw": base_dir/"Bronze"/"QBO"/"Raw"/f"{self.today.year}_{self.today.month}",
                "GL": base_dir/"Bronze"/"QBO"/"GeneralLedger",
                "PL": base_dir/"Bronze"/"QBO"/"ProfitAndLoss",
                "Time":base_dir/"Bronze"/"QBOTime"
            },
            "Delivery": {"Traction":base_dir/"Bronze"/"Traction", "HP":base_dir/"Bronze"/"HarvestProfit"},
            "Auth": {"QBO":base_dir/"Bronze"/"Authentication"/"QBO", "QBOTime": base_dir/"Bronze"/"Authentication"/"QBOTime"},
            "Log": base_dir/"Load_History"/f"{self.today.year}"/month_format
        }
        self.silver_path = {
            "QBO": {
                "Dimension_time": base_dir/"Silver"/"QBO"/"Dimension"/f"{self.today.year}_{self.today.month}",
                "Dimension": base_dir/"Silver"/"QBO"/"Dimension",
                "Raw": base_dir/"Silver"/"QBO"/"Fact"/"Raw",
                "PL": base_dir/"Silver"/"QBO"/"Fact"/"ProfitAndLoss",
                "GL": base_dir/"Silver"/"QBO"/"Fact"/"GeneralLedger",
                "Time": base_dir/"Silver"/"QBOTime"
            },
            "Delivery": {"Traction":base_dir/"Silver"/"Traction", "HP":base_dir/"Silver"/"HarvestProfit"}
        }
        
    
    def get_fx(self):
        key  = os.getenv("ALPHAVANTAGE_KEY")
        url  = ("https://www.alphavantage.co/query?"
                "function=CURRENCY_EXCHANGE_RATE"
                "&from_currency=USD&to_currency=CAD"
                f"&apikey={key}")
        rate = float(requests.get(url, timeout=10).json()
                    ["Realtime Currency Exchange Rate"]["5. Exchange Rate"])
        self.fx = rate
    
    def create_log(self, path: Path) -> None:
        self.check_file(path)
        day_format = "".join(["0",str(self.today.day)]) if self.today.day < 10 else str(self.today.day)
        self.log = open(path/(day_format+"_Log.txt"), "a")

    
    def close_log(self):
        self.log.close()

    def check_file(self, path: Path) -> None:
        if not Path.exists(path):
            os.makedirs(path)
    
    def formulate_date(self, df:pd.DataFrame, date_cols:list[str], drop_time:bool=True) -> pd.DataFrame:
        """ 
            format the date columns into datetime format
        """
        assert len(set(date_cols) - set(df.columns)) == 0, "Not all columns passed are inside dataframe passed"
        for col in date_cols:
            df[col] = pd.to_datetime(df[col], utc=True)
            if drop_time: df[col] = df[col].dt.date
        return df





In [151]:
class Projects(Job):
    """ 
        for project specific data transformations
    """
    
    def __init__(self, focus_last_FY:bool = False):
        super().__init__()
        self.gold_path = {
            "weekly_banking": self.base_dir / "Gold" / "FinanceProject" / "WeeklyBanking",
            "inventory": self.base_dir / "Gold" / "InventoryProject",
            "payroll": self.base_dir / "Gold" / "HRProject" /"PayrollProject",
            "finance_operational": self.base_dir / "Gold" / "FinanceOperationalProject",
            "budget": self.base_dir / "Gold" / "BudgetProject",
            "QBOTime": self.base_dir / "Gold" / "HRProject" / "QBOTimeProject",
            "hr_combined": self.base_dir / "Gold" / "HRProject" / "CombinedSummary"
        }
        self.silver_acc = pd.read_csv(self.silver_path["QBO"]["Dimension_time"]/"Account.csv")
        self.commodities = {
            "Produce": ["Strawberry", "Watermelon", "Cantaloupe", "Market Garden", "Broccoli", "Pumpkin", "Sweet Corn", "Cauliflower", "Squash", "Honeydew Melon", "Potato", "Carrot", "Cabbage",
                        "Lettuce", "Brussel Sprouts", "Prairie Pathways", "Beet", "Corn Maze", "CSA"],
            "Grain": ["Blackeye Pea", "Winter Wheat", "Durum", "Cotton", "Chickpea", "Barley", "Green Lentil", "Red Lentil", "Canola", 
                        "Wheat","Field Pea", "Corn", "Oat", "Soybean", "Bean"],
            "Cattle": ["Weaned Calves", "Cull Bull", "Cull Cow", "Bred Heifer", "Purebred Yealing Bull", "Purebred Heifer", 
                        "Purebred Cow", "Purebred Bull", "Cow", "Bull", "Steer", "Heifer", "Yearling", "Calf"]
        }
        self.locations = {
            "Produce": ["BritishColumbia (produce)", "Outlook", "Arizona (produce)"],
            "Cattle": ["Airdrie", "Eddystone (cattle)", "Ashcroft", "Home Ranch", "Diamond S", "Wolf Ranch", "Fraser River Ranch", "Moon Ranch", "Waldeck", "Calderbank"],
            "Grain": ["Eddystone (grain)", "Arizona (grain)", "Colorado", "Swift Current", "Regina", "Raymore", "Prince Albert", "The Pas",
                      "Kamsack", "Hafford", "Yorkton", "Fly Creek", "Camp 4", "Havre", "Billings"],
            "Seed": ["NexGen", "Seeds", "Seeds USA"],
            "Others": ["Eddystone (corporate)", "Arizona (corporate)", "Legacy", "BritishColumbia (corporate)", "-Corporate"]
        }
        self.bc_ranches = ["Ashcroft", "Fraser River Ranch", "Moon Ranch", "Wolf Ranch", "Home", "Diamond S", "BritishColumbia (corporate)","Home Ranch"]
        self.pl_exist = False # determines whether _financial_operational has run and gold_pl is stored in self, if not, any subsequent downstream projects will run _financial_operational first
        self.currentFY = self.today.year if self.today.month<=10 else self.today.year + 1
        if focus_last_FY: self.currentFY -= 1

    def _pillar_classification(self, entry: pd.Series) -> str:
        """ 
            this function classifies pillar of a transaction based on location
        """
        location = entry["Location"]
        if not isinstance(location, str):
            return "Missing"
        if "produce" in location:
            return "Produce"
        elif "grain" in location:
            return "Grain"
        elif "cattle" in location:
            return "Cattle"
        elif "corporate" in location:
            return "Unclassified"
        match location.lower():
            case "hafford"|"kamsack"|"prince albert"|"raymore"|"regina"|"swift current"|"the pas"|"camp 4"|"fly creek"|"havre"|"yorkton"|"colorado"|"billings":
                return "Grain"
            case "outlook"|"seeds usa":
                return "Produce"
            case "ashcroft"|"diamond s"|"fraser river ranch"|"home ranch"|"moon ranch"|"wolf ranch"|"waldeck"|"calderbank"|"airdrie":
                return "Cattle"
            case "seeds"|"nexgen":
                return "Seed"
            case _:
                return "Unclassified"
    
    def _identify_product(self, entry: pd.Series, for_budget:bool=False) -> str:
        """ 
            this function identifies commodity from account names, except for seed, 
                if this function is called from budget project, it combines MG & CSA and take CM into consideration, and name forage differently
        """
        if not for_budget:
            if entry["Corp"] in ["MSL", "MSUSA", "NexGen"]:
                return "SeedProduct"
        accname = entry["AccName"].lower() if not for_budget else entry["AccFull"].lower()
        if "float" in accname:
            return "Others"
        for x in self.commodities["Produce"] + self.commodities["Grain"] + self.commodities["Cattle"]:
            if x.lower() in accname:
                if for_budget:
                    match x:
                        case "Market Garden"|"CSA":
                            return "Market Garden / CSA"
                        case "Corn Maze":
                            return "Prairie Pathways"
                    return x 
                return x
        if "straw" in accname or "forage" in accname or "hay bale" in accname:
            if for_budget: 
                return "Hay/Silage" 
            else: 
                return "Forage"
        return "Others"
    
    def _weekly_banking(self) -> None:
        """ 
            weekly banking project: match latest GL bank transactions with raw activities - extract accounts for those activities
                assumptions: a raw entry (e.g., invoice) can have multiple lines - multiple associated accounts, only considering the first one 
        """
        print("\nStarting Weekly Banking Project Transformation\n")
        # determine minal date to keep for GL
        if self.today.month > 6:
            year = self.today.year 
            month = self.today.month - 6 
        else:
            year = self.today.year - 1
            month = self.today.month + 12 - 6
        # load and prepare data
        ## account
        account = self.silver_acc.copy(deep=True)
        ## change some accounts to Transfer category
        acc_list = ["MFL264", "MSL250"]
        account.loc[account["AccID"].isin(acc_list), "Profitem"] = "Asset"
        account.loc[account["AccID"].isin(acc_list), "Category"] = "Transfer"
        account_bank = account[account["AccountType"]=="Bank"]
        ## LinkedTxn for invoice and bill
        invoice_linked = pd.read_csv(self.silver_path["QBO"]["Raw"] / "LinkedTxn"/ "LinkedTxn_Mapping_Invoice.csv")
        bill_linked = pd.read_csv(self.silver_path["QBO"]["Raw"] / "LinkedTxn"/ "LinkedTxn_Mapping_Bill.csv")
        mapping = pd.concat([invoice_linked, bill_linked])
        mapping = mapping.drop(columns=["Corp"])
        # define customized function for processing other raw table
        def _process_facts(df_type:str) -> pd.DataFrame:
            """ 
                function for processing raw tables for mapping table - TransactionID_partial to AccID
            """
            df = pd.read_csv(self.silver_path["QBO"]["Raw"]/(df_type+".csv"), usecols = ["TransactionID", "AccID"])
            df["TransactionID"] = df["TransactionID"].apply(lambda x: x.split("-")[1])
            df = df.drop_duplicates()
            df = df.rename(columns={"TransactionID":"TxnId"})
            return df
        ## purchase table for expense transactions
        purchase = _process_facts("Purchase")
        purchase["TxnType"] = "Expense"
        mapping = pd.concat([mapping,purchase])
        ## journal entries - exclude most entries related to bank
        journal = _process_facts("JournalEntry")
        journal["TxnType"] = "Journal Entry"
        # for journal entries, exclude most of entires where the activity account ID is a bank ID
        exclude_list = list(account_bank.AccID.unique())
        # mylist = ["MFL51", "MFBC470", "MFBC471", "MFL28", "MFL27", "MFL1150040024"]
        mylist = ["MFBC470", "MFBC471"] # should include these accounts
        for acc in mylist:
            exclude_list.remove(acc)
        journal = journal[~journal["AccID"].isin(exclude_list)]
        mapping = pd.concat([mapping,journal])
        ## deposit
        deposit = _process_facts("Deposit")
        deposit["TxnType"] = "Deposit"
        mapping = pd.concat([mapping,deposit])
        ## salesreceipts
        sales = _process_facts("SalesReceipt")
        sales["TxnType"] = "Sales Receipt"
        mapping = pd.concat([mapping,sales])
        # process mapping table - dedup
        mapping = mapping.drop_duplicates(subset=["TxnId"],keep="first")
        ## load GL transacitons
        cols = ["TransactionType","TransactionID_partial","AccID","AccNum","AccName", "TransactionDate", "Amount", "SplitAcc", "SplitAccID", "Memo", "Corp", "Balance"]
        transactions = pd.read_csv(self.silver_path["QBO"]["GL"]/"GeneralLedger.csv",dtype={"TransactionID_partial":str}, usecols=cols)
        transactions = transactions[transactions["AccID"].isin(account_bank.AccID.unique())]
        transactions["TransactionDate"] = pd.to_datetime(transactions["TransactionDate"])
        transactions = transactions[transactions["TransactionDate"]>=dt.datetime(year, month, 1)]
        transactions = transactions.rename(columns={"TransactionType":"TxnType","TransactionID_partial":"TxnId",
                                                    "AccID":"BankAccID","AccNum":"BankAccNum","AccName":"BankAccName",
                                                    "TransactionDate":"BankActivityDate","Amount":"BankAmount"})
        # merge to get CurrencyID for bank_acc
        transactions = pd.merge(transactions, account_bank.loc[:,["AccID","CurrencyID"]], left_on=["BankAccID"], right_on=["AccID"], how="left")
        transactions = transactions.drop(columns=["AccID"])
        # separating transfers - don't merge with mapping table
        transfers = transactions[transactions["TxnType"] == "Transfer"].copy(deep=True)
        transactions = transactions[transactions["TxnType"]!="Transfer"]
        transactions = transactions.drop(columns=["SplitAcc", "SplitAccID"])
        transactions["BankActivityDate"] = pd.to_datetime(transactions["BankActivityDate"])
        transactions["TxnType"] = transactions["TxnType"].replace({"Cheque Expense":"Expense", "Check": "Expense"})
        # merge with mapping table
        transactions_mapped = pd.merge(transactions,mapping,on=["TxnId","TxnType"],how="left")
        non_match = transactions_mapped[transactions_mapped["AccID"].isna()]
        print("None Match Transaction Types")
        print(non_match.TxnType.value_counts())
        print(f"Non matches - {len(non_match)}")
        # function to determine transfer type
        def _determine_transfer_type(entry:str) -> str:
            """ 
                determine whether the transfer is for visa, bank, or other transfer
            """
            if "visa" in entry.lower():
                return "Visa Payment"
            elif "due" in entry.lower():
                return "Bank Transfer"
            else:
                return "Other Transfer"
        # allocate transfer type 
        transfers["TransferType"] = transfers["SplitAcc"].apply(lambda x: _determine_transfer_type(x))
        transfers = transfers.rename(columns={"SplitAccID":"AccID"})
        transfers = transfers.drop(columns=["SplitAcc"])
        transactions_mapped = pd.concat([transactions_mapped,transfers], ignore_index=True)
        # clean up the dataframe
        transactions_mapped = transactions_mapped.rename(columns={"CurrencyID":"BankCurrencyID"})
        transactions_mapped = pd.merge(transactions_mapped, account.loc[:,["AccID","AccName","AccNum","Category","Profitem","CurrencyID"]], on="AccID", how="left")
        transactions_mapped.loc[transactions_mapped["TransferType"]=="Bank Transfer","Category"] = "Bank Transfer"
        transactions_mapped.loc[((transactions_mapped["BankAccNum"].str.startswith("MSL"))&(transactions_mapped["AccNum"]=="MSL120001")), "Category"] = "Seed Processing Revenue"
        transactions_mapped = transactions_mapped.rename(columns={"AccNum":"ActivityAccNum", "AccName":"ActivityAccName"})
        # csv from sharepoint is unstable, and produced unpredictable readings from Power BI
        self.check_file(self.gold_path["weekly_banking"])
        transactions_mapped.to_excel(self.gold_path["weekly_banking"]/"BankingActivity.xlsx", sheet_name="transactions", index=False)

    def _finance_operational(self) -> None:
        """ 
            transform PL data into operational-ready
                1. reclassify accounts
                2. standardize location, classify pillar
                3. revising signs
        """
        print("\nStarting Finance Operational Project Transformation\n")
        # load data from silver space
        data = pd.read_csv(self.silver_path["QBO"]["PL"]/"ProfitAndLoss.csv")
        assert len(data.FXRate.value_counts()) == 1, "different FXRate detected"
        self.fx = data.loc[0,"FXRate"]
        data["TransactionDate"] = pd.to_datetime(data["TransactionDate"])
        data["FiscalYear"] = data.TransactionDate.apply(lambda x: x.year + 1 if x.month >= 11 else x.year)
        # add month to the PL
        data["Month"] = data["TransactionDate"].dt.month_name()
        ## add location for seed operation
        data.loc[data["Corp"]=="MSL","Location"] = "Seeds"
        data.loc[data["Corp"]=="NexGen","Location"] = "NexGen"
        data.loc[data["Corp"]=="MSUSA","Location"] = "Seeds USA"
        # clean location
        data = data.rename(columns={"Location":"LocationRaw"})
        data["Location"] = data["LocationRaw"]
        # switch seeds usa to AZ produce
        data.loc[data["Corp"]=="MSUSA","Location"] = "Arizona (produce)"
        ## clean location
        clean_location = {"Airdrie - Grain":"Airdrie", "Airdrie - Cattle":"Airdrie", "Airdrie - General":"Airdrie", "Airdrie":"Airdrie", 
                        "Eddystone - Grain": "Eddystone (grain)", "Eddystone - Cattle": "Eddystone (cattle)", "Eddystone - General":"Eddystone (corporate)",
                        "Outlook (JV)":"Outlook", "AZ Produce":"Arizona (produce)", "Corporate":"Arizona (corporate)", "BC Produce":"BritishColumbia (produce)",
                        "Grain":"Arizona (grain)", "Cache/Fischer/Loon - DNU":"Legacy", "Ashcroft (CC, Fischer, Loon)":"Ashcroft", 
                        "Outlook (Capital)":"Outlook", "Colorado (MF)":"Colorado", "Colorado (JV)":"Colorado", "Cattle - General":"BritishColumbia (corporate)",
                        "Home (70 M, LF/W, 105 M)":"Home Ranch", "Diamond S (BR)":"Diamond S", "North Farm (deleted)":"Legacy"}
        data["Location"] = data["Location"].replace(clean_location)
        locations = self.locations["Produce"] + self.locations["Grain"] + self.locations["Cattle"] + self.locations["Others"] + self.locations["Seed"]
        unaccounted_location = list(set(data["Location"].unique()) - set(locations))
        print(f"location unaccounted for - {unaccounted_location}")
        # classify pillar
        data["Pillar"] = data.apply(lambda x: self._pillar_classification(x),axis=1)
        # reorganize corp
        ## MPUSA missing location = Arizona (produce)
        data.loc[((data["Corp"] == "MPUSA")&(data["Location"].isna())), "Location"] = "Arizona (produce)"
        data.loc[((data["Corp"] == "MPUSA")&(data["Location"] == "Arizona (produce)")), "Pillar"] = "Produce"
        ## AZ Produce --> MPUSA
        data.loc[data["Location"] == "Arizona (produce)", "Corp"] = "MPUSA"
        ## move everything for AZ in 2025 to produce
        data.loc[((data["FiscalYear"] >= 2025) & (data["Location"].str.contains("Arizona",case=False))),"Pillar"] = "Produce"
        data.loc[((data["FiscalYear"] >= 2025) & (data["Location"].str.contains("Arizona",case=False))),"Location"] = "Arizona (produce)"
        ## BC Produce --> MPL
        data.loc[data["Location"] == "BritishColumbia (produce)", "Corp"] = "MPL"
        ## Outlook --> MPL
        data.loc[data["Location"]=="Outlook", "Corp"] = "MPL"
        # Reclassify accounts for Operational Purpose
        ## read & process operational classification
        with open(self.silver_path["QBO"]["Dimension"]/"acc_classification.yaml", "r", encoding="utf-8") as f:
            raw_acc = yaml.safe_load(f)
        rows = [(l1, l2, l3, v) 
                for l1, l1_inner in raw_acc.items() 
                for l2, l2_inner in l1_inner.items() 
                for l3, l3_inner in l2_inner.items() 
                for v in l3_inner]
        acc_operation = pd.DataFrame(rows, columns=["OperationProfType", "OperationCategory", "OperationSubCategory", "AccID"])



        # acc_operation = pd.read_csv(self.silver_path["QBO"]["Dimension"]/"Accounts Classification - Operation.csv", dtype={"Pillar": str, "IsGenric": str}, keep_default_na=False)
        # acc_operation = acc_operation.rename(columns={"Pillar":"AccPillar", "OperationProfiType":"OperationProfType"})
        # acc_operation["AccNum"] = acc_operation["AccountName"].apply(lambda x: x.split(" ")[0])
        # acc_operation["IsIntercompany"] = acc_operation["AccountName"].apply(lambda x: "Yes" if "intercompany" in x.lower() else "No")
        ## classify commodity
        # acc_operation["Commodity"] = acc_operation.apply(lambda x: self._identify_product(x), axis=1)
        # acc_operation.to_csv(self.gold_path["finance_operational"]/"accounts_classified_operation.csv", index=False)
        ## read accounts table and apply new classification
        accounts = self.silver_acc
        # accounts1 = accounts[accounts["AccNum"].isna()].copy(deep=True)
        # accounts = accounts[accounts["AccNum"].notna()]
        accounts = pd.merge(accounts, acc_operation, on = "AccID", how = "left")
        accounts["Commodity"] = accounts.apply(lambda x: self._identify_product(x), axis=1)
        self.operation_acc = accounts[accounts["AccNum"].notna()]   # consider only accounts with account number for budget
        self.operation_acc = self.operation_acc[self.operation_acc["Active"]] # only look at active accounts for budget
        self.operation_acc.to_csv(self.gold_path["finance_operational"]/"Account_table_budget.csv", index=False)
        # accounts = pd.concat([accounts,accounts1],ignore_index=True)
        # Revising Signs according to Operational Classification
        print("Revising Signs ...")
        expense_accounts = accounts[(accounts["OperationCategory"] == "Expense") | (accounts["OperationCategory"] =="Inventory Consumption")]
        data["AmountDisplay"] = data.apply(lambda x: -x["AmountCAD"] if x["AccID"] in expense_accounts.AccID.unique() else x["AmountCAD"], axis=1)
        self.gold_pl = data
        self.gold_acc = accounts
        # save files
        print("Saving ...")
        self.check_file(self.gold_path["finance_operational"])
        data.to_csv(self.gold_path["finance_operational"]/"PL.csv", index=False)
        accounts.to_excel(self.gold_path["finance_operational"]/"Account_table.xlsx", sheet_name = "Account", index=False)
        data.to_excel(self.gold_path["finance_operational"]/"PL.xlsx", sheet_name="Transactions", index=False)
        self.pl_exist = True

    def _process_pp(self, data:pd.DataFrame) -> pd.DataFrame:
        """ 
            This function takes original dataframe, apply the payperiod number classification based on transactions date, process payperiod columns, and return the new dataframe,
                save the pp table for consolidated tables
        """
        # load payperiods
        payperiods = pd.read_csv(self.gold_path["payroll"]/"Payperiods.csv")
        payperiods["START"] = pd.to_datetime(payperiods["START"])
        payperiods["END"] = pd.to_datetime(payperiods["END"])
        payperiods = payperiods.loc[:,["PP","START","END","Cycle","FiscalYear"]]
        def _determine_pp(entry:pd.Series, date_col:str = "TransactionDate") -> str:
            """ 
                This function determined which payperiod a transaction should be classified into, 
                    starting from most recent payperiod, the algorithm uses period start date + drift to determine which payperiod a transaction should fall into
                Assumptions:
                    1. for outlook and az, each payperiod is shifted by 5 + 7 days forward
                    2. for other location, each payperiod is shifted by 5 days forward
            """
            date = entry[date_col] 
            if isinstance(entry["Location"],str):
                location = entry["Location"].lower()
            else:
                location = "None"
            if "outlook" in location or "az" in location or "arizona" in location:
                date_diff = dt.timedelta(days=5+7)
            else:
                date_diff = dt.timedelta(days=5)
            year = date.year 
            month = date.month
            # push back the most recent payperiod dates for older transactions to save compute
            if month >= 11:
                payperiods_subset = payperiods[payperiods["END"] <= dt.datetime(year+1,2,1)] 
            else:  
                payperiods_subset = payperiods[payperiods["END"] <= dt.datetime(year,month+2,1)]
            for i in range(len(payperiods_subset)-1,-1,-1):
                if date > (payperiods_subset.loc[i,"END"]+date_diff):
                    return "Exceed Max PayPeriod"
                if date >= (payperiods_subset.loc[i,"START"]+date_diff):
                    return str(payperiods_subset.loc[i,"PP"]) + "-" + str(payperiods_subset.loc[i,"Cycle"]) + "-" + str(payperiods_subset.loc[i,"FiscalYear"])
            return "Earlier than Min PayPeriod"
        print("Allocating PPNum for transactions ...")
        date_col = "TransactionDate" if "TransactionDate" in data.columns else "date"
        data["PPNum"] = data.apply(lambda x: _determine_pp(x,date_col),axis=1)
        data = data[data["PPNum"] != "Earlier than Min PayPeriod"].copy(deep=True).reset_index(drop=True) # eliminate earlier than min payperiod in the csv, note dates are shifted in the csv
        data["Cycle"] = data["PPNum"].apply(lambda x: x.split("-")[1])
        data["FiscalYear"] = data["PPNum"].apply(lambda x: int(x.split("-")[2]))
        data["PPNum"] = data["PPNum"].apply(lambda x: x.split("-")[0])
        data["PPName"] = data["PPNum"].apply(lambda x: "PP0" + x if int(x) < 10 else "PP" + x)
        data["PPName"] = data["Cycle"].str.slice(2,) + "-" + data["PPName"]
        data.loc[:,["PPName", "PPNum", "Cycle", "FiscalYear"]].drop_duplicates().to_csv(self.gold_path["payroll"].parent/ "OtherTables" / "PayPeriods.csv", index=False)
        return data

    def _process_units(self) -> None:
        """ 
            this function read and process Unit files that contains unit numbers for each location
        """
        acres = pd.read_csv(self.gold_path["payroll"]/"Unit.csv",dtype={"Location":str, "Unit":float})
        acres["Location"] = acres["Location"].str.strip()
        doc_rename = {"Airdrie Grain": "Airdrie (grain)", "Aridrie Cattle (head days 365)":"Airdrie", "Arizona All":"Arizona (produce)",
                    "BC Cattle (head days 365)":"BritishColumbia (cattle)", "BC Produce":"BritishColumbia (produce)", 
                    "Box Elder":"Havre", "Eddystone Cattle (head days 365)":"Eddystone (cattle)", "Eddystone Grain":"Eddystone (grain)",
                    "Monette Seeds CDN (avg met. ton)":"Seeds", "Monette Seeds USA":"Seeds USA", "NexGen (avg met. ton)":"NexGen",
                    "Waldeck (head days 365)":"Waldeck", "Calderbank  (head days 365)":"Calderbank"}
        acres["Location"] = acres["Location"].replace(doc_rename)
        acres["Pillar"] = acres.apply(lambda x: self._pillar_classification(x),axis=1)
        acres.to_csv(self.gold_path["payroll"].parent/ "OtherTables" /"Unit_PowerBI.csv",index=False)

    def _payroll_project(self) -> None: 
        """ 
            will run _finance_operational() first
            output: details + cost per unit (units per location input sheet) + average cost per unit for FY
        """
        self.check_file(self.gold_path["payroll"].parent/ "OtherTables")
        if not self.pl_exist:
            self._finance_operational()
        print("\nStarting Payroll Project Transformation\n")

        # load and filter accounts for wages and contract labor
        account = self.silver_acc[(self.silver_acc["Category"].isin(["Wages and benefits - direct","Wages and benefits - overhead"]) | (self.silver_acc["AccNum"].isin(["MFAZ595001","MFBC536030"])))] 
        # load only with transaction date later than 2021-12-20, and without "Accrual" in the memo
        data = self.gold_pl.copy(deep=True)
        data = data[data["AccID"].isin(account.AccID.unique())]
        data["TransactionDate"] = pd.to_datetime(data["TransactionDate"])
        data = data[data["TransactionDate"]>=dt.datetime(2021,12,20)].reset_index(drop=True)
        data = data[~data["Memo"].str.contains("Accrual",case=False,na=False)]
        # allocating payperiods
        data = self._process_pp(data=data)
        # standardizing location
        # data.loc[data["Location"]=="Airdrie (corporate)", "Pillar"] = "Cattle"                # deprecated
        # data.loc[data["Location"]=="Airdrie (corporate)", "Location"] = "Airdrie (cattle)"    # deprecated
        data.loc[data["Location"]=="Eddystone (corporate)", "Pillar"] = "Unclassified"
        data.loc[data["Location"]=="Eddystone (corporate)", "Location"] = "Unassigned"
        data.loc[data["Location"]=="Legacy", "Location"] = "Unassigned"
        data.loc[(data["Location"].str.contains("corporate",case=False,na=False)&(data["Location"]!="BritishColumbia (corporate)")),"Location"] = "Corporate"
        ## move BC ranches into BC Cattle
        data.loc[(data["Location"].isin(self.bc_ranches)), "Location"] = "BritishColumbia (cattle)"
        data.loc[data["Location"] == "BritishColumbia (cattle)", "Pillar"] = "Cattle"
        # summarizing data
        ## by Location per PP
        data_summarized = pd.DataFrame(data.groupby(["Location","PPName","Pillar","FiscalYear","Cycle","PPNum"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False))
        assert len(data_summarized) == len(data.groupby(["Location","PPName"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)), "Duplicated value detected for per Location per PP calculation"
        ## join acres data for CostPerUnit compute
        print("Summarizing ...")
        self._process_units()
        acres = pd.read_csv(self.gold_path["payroll"].parent/ "OtherTables" /"Unit_PowerBI.csv",dtype={"Location":str, "Unit":float})
        acres = acres.loc[:,["Location", "Unit"]]
        print(f"Unaccounted location for Acres Doc: {set(acres.Location.unique()) - set(data_summarized.Location.unique())}")
        data_summarized = pd.merge(data_summarized, acres, on="Location", how="left")
        data_summarized["CostPerUnit"] = data_summarized["AmountDisplay"] / data_summarized["Unit"] * 26
        data_summarized["Count"] = 1
        ## by Location
        data_summarized2 = data_summarized.groupby(by=["Location","FiscalYear","Pillar"]).agg({"CostPerUnit":"mean", "Count":"sum"}).reset_index(drop=False)
        data_summarized2 = data_summarized2.rename(columns={"CostPerUnit":"Avg CostPerUnit"})
        assert len(data_summarized2) == len(data_summarized.groupby(by=["Location","FiscalYear"]).agg({"CostPerUnit":"mean"})), "Duplicated value detected for per Location calculation"
        ## by pillar
        data_summarized3 = data_summarized2.groupby(by=["FiscalYear","Pillar"]).agg({"Avg CostPerUnit":"mean", "Count":"sum"}).reset_index(drop=False)
        assert len(data_summarized3) == len(data_summarized.groupby(by=["Pillar","FiscalYear"]).agg({"CostPerUnit":"mean"})), "Duplicated value detected for per Pillar calculation"
        # saving
        print("Saving ...")
        self.check_file(self.gold_path["payroll"])
        data.to_excel(self.gold_path["payroll"]/"Payroll.xlsx", sheet_name="Payroll", index=False)
        self.check_file(self.gold_path["hr_combined"] / "CSV")
        data_summarized.to_csv(self.gold_path["hr_combined"]/ "CSV" / "payroll_summarized1.csv", index=False)
        data_summarized2.to_csv(self.gold_path["hr_combined"]/ "CSV" / "payroll_summarized2.csv", index=False)
        data_summarized3.to_csv(self.gold_path["hr_combined"]/ "CSV" / "payroll_summarized3.csv", index=False)

    def _QBOTime_project(self) -> None:
        """ 
            apply PP allocation to QBO Time data, clean locaiton, and join relevant info into one table
        """
        print("\nStarting QBO Time Project Transformation\n")
        # read files
        timesheets = pd.read_csv(self.silver_path["QBO"]["Time"]/"timesheets.csv")
        jobcode = pd.read_csv(self.silver_path["QBO"]["Time"]/"jobcodes.csv")
        users = pd.read_csv(self.silver_path["QBO"]["Time"]/"users.csv")
        group = pd.read_csv(self.silver_path["QBO"]["Time"]/"group.csv")
        print(f"Read {len(timesheets)} timesheet records, {len(jobcode)} jobcodes, {len(users)} users, {len(group)} groups")
        timesheets_len, users_len = len(timesheets), len(users)
        # clean up location in group table
        ## Arizona - all produce
        group.loc[((group["corp_short"]=="A")&(group["location_name"]=="Monette Farms AZ")), "Location"] = "Arizona (produce)"
        group.loc[((group["corp_short"]=="A")&(group["location_name"]=="Monette Produce USA")), "Location"] = "Arizona (produce)"
        group.loc[((group["corp_short"]=="A")&(group["location_name"]=="Monette Seeds USA")), "Location"] = "Arizona (produce)"
        ## BC
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Ashcroft Ranch")), "Location"] = "Ashcroft"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Cache/Fischer/Loon")), "Location"] = "BritishColumbia (cattle)"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"].str.contains("silage", case=False))), "Location"] = "BritishColumbia (cattle)"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Diamond S Ranch")), "Location"] = "Diamond S"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Fraser River Ranch")), "Location"] = "Fraser River Ranch"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Home Ranch (70 Mile, LF/W, BR)")), "Location"] = "Home Ranch"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Moon Ranch")), "Location"] = "Moon Ranch"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Produce")), "Location"] = "BritishColumbia (produce)"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="Wolf Ranch")), "Location"] = "Wolf Ranch"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="SAWP")), "Location"] = "BritishColumbia (produce)"
        group.loc[((group["corp_short"]=="BC")&(group["location_name"]=="SAWP Produce")), "Location"] = "BritishColumbia (produce)"
        ## Outlook
        group.loc[((group["corp_short"]=="O")), "Location"] = "Outlook"
        ## others
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Airdrie")), "Location"] = "Airdrie"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="BC")), "Location"] = "Unassigned"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Calderbank")), "Location"] = "Calderbank"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Eddystone")), "Location"] = "Eddystone (unspecified)"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Hafford")), "Location"] = "Hafford"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Kamsack")), "Location"] = "Kamsack"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="MFUSA Billings")), "Location"] = "Billings"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="MFUSA Box Elder")), "Location"] = "Havre"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Nexgen Seeds")), "Location"] = "NexGen"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Prince Albert")), "Location"] = "Prince Albert"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Raymore")), "Location"] = "Raymore"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Regina")), "Location"] = "Regina"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Russel Approvals")), "Location"] = "Unassigned"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Seeds")), "Location"] = "Seeds"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Swift Current")), "Location"] = "Swift Current"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="The Pas")), "Location"] = "The Pas"
        group.loc[((group["corp_short"]=="CM")&(group["location_name"]=="Waldeck")), "Location"] = "Waldeck"
        unclassified = group[group["Location"].isna()].location_name.unique()
        if len(unclassified) > 0: print(f"\nUnclassified location - {unclassified}\n")
        # create another location column for general location where bc ranches are merged into one
        group = group.rename(columns={"Location": "Location (detail)"})
        group["Location"] = group["Location (detail)"]
        group.loc[(group["Location (detail)"].isin(self.bc_ranches)), "Location"] = "BritishColumbia (cattle)"
        # merge tables into one table
        ## merge location into users
        users = pd.merge(users, group.loc[:,["group_id", "location_name", "Location", "Location (detail)"]].drop_duplicates(), on="group_id", how="left")
        ## merge users into timesheets
        timesheets = pd.merge(timesheets,users.loc[:,["user_id", "group_id", "username", "full_name", "location_name","Location","Location (detail)"]], on="user_id", how="left")
        ## merge job into timesheets
        timesheets = pd.merge(timesheets, jobcode.loc[:,["jobcode_id","job_name","type"]].rename(columns={"type":"job_type"}), on="jobcode_id", how="left")
        assert (len(users) == users_len) and (len(timesheets) == timesheets_len), f"duplicated records found, timesheets - {timesheets_len} vs {len(timesheets)}; users - {users_len} vs {len(users)}"
        # classify payperiods
        timesheets["date"] = pd.to_datetime(timesheets["date"])
        timesheets = self._process_pp(data=timesheets)
        # modify location for BC0
        timesheets.loc[timesheets["user_id"] == "BC6107856", "Location"] = "Unassigned"
        # classify pillars
        timesheets["Pillar"] = timesheets.apply(lambda x: self._pillar_classification(x), axis=1)
        timesheets.loc[timesheets["Pillar"] == "Missing", "Pillar"] = "Unclassified"
        # summarizing data
        ## by Location per PP 
        summarized = timesheets.groupby(["Location","PPName","FiscalYear","Cycle","PPNum", "Pillar"]).agg({"duration":"sum"}).reset_index(drop=False)
        assert len(summarized) == len(timesheets.groupby(["Location","PPName"]).agg({"duration":"sum"})), "duplicated value detected for timsheet per Location per PP summarization"
        ## read units file
        acres = pd.read_csv(self.gold_path["payroll"].parent/ "OtherTables" /"Unit_PowerBI.csv",dtype={"Location":str, "Unit":float})
        acres = acres.loc[:,["Location", "Unit"]]
        addition = pd.DataFrame(data={"Location":["Billings"], "Unit":[acres[acres["Location"].isin(['Fly Creek', 'Camp 4'])].Unit.sum()]})
        acres = pd.concat([acres,addition],ignore_index=True)
        print(f"Unaccounted location for Acres Doc: {set(acres.Location.unique()) - set(summarized.Location.unique())}")
        print(f"Unaccounted location for timesheets: {set(summarized.Location.unique()) - set(acres.Location.unique())}")
        ## merge with units file
        summarized = pd.merge(summarized, acres, on="Location", how="left")
        ## calculate hours per unit
        summarized["HoursPerUnit"] = summarized["duration"] / summarized["Unit"] * 26
        summarized["Count"] = 1
        # summarize per location
        summarized2 = summarized.groupby(by=["Location","FiscalYear", "Pillar"]).agg({"HoursPerUnit":"mean", "Count":"sum"}).reset_index(drop=False)
        summarized2 = summarized2.rename(columns={"HoursPerUnit":"Avg HoursPerUnit"})
        assert len(summarized2) == len(timesheets.groupby(["Location","FiscalYear"]).agg({"duration":"sum"})), "duplicated value detected for timsheet per Location summarization"
        # summarize per pillar
        summarized3 = summarized2.groupby(by=["FiscalYear", "Pillar"]).agg({"Avg HoursPerUnit":"mean", "Count":"sum"}).reset_index(drop=False)
        assert len(summarized3) == len(timesheets[timesheets["Pillar"]!="Missing"].groupby(["Pillar","FiscalYear"]).agg({"duration":"sum"})), "duplicated value detected for timsheet per Pillar summarization"

        # saving
        print("Saving ...\n")
        self.check_file(self.gold_path["QBOTime"])
        timesheets.to_excel(self.gold_path["QBOTime"]/"QBOTime.xlsx", sheet_name = "QBOTime", index=False)
        self.check_file(self.gold_path["hr_combined"]/ "CSV")
        summarized.to_csv(self.gold_path["hr_combined"]/ "CSV" / "time_summarized1.csv", index=False)
        summarized2.to_csv(self.gold_path["hr_combined"]/ "CSV" / "time_summarized2.csv", index=False)
        summarized3.to_csv(self.gold_path["hr_combined"]/ "CSV" / "time_summarized3.csv", index=False)

    def _hr_summary(self) -> None:
        """ 
            This function consolidate payroll and QBO time summaries into one table for consolidated insights
        """
        final_df = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
        for i in [1, 2, 3]:
            payroll = pd.read_csv(self.gold_path["hr_combined"] / "CSV" / f"payroll_summarized{i}.csv")
            payroll_rename = {"AmountDisplay": "TotalAmount", "CostPerUnit": "AmountPerUnit", "Avg CostPerUnit": "Avg AmountPerUnit"}
            payroll = payroll.rename(columns=payroll_rename)
            payroll["Mode"] = "Payroll"
            time = pd.read_csv(self.gold_path["hr_combined"] / "CSV" / f"time_summarized{i}.csv")
            time_rename = {"duration": "TotalAmount", "HoursPerUnit": "AmountPerUnit", "Avg HoursPerUnit": "Avg AmountPerUnit"}
            time = time.rename(columns=time_rename)
            time["Mode"] = "Hours"
            final_df[i-1] = pd.concat([payroll, time], ignore_index=True)
        final_df[0].to_excel(self.gold_path["hr_combined"]/"Summarized.xlsx", sheet_name="Summarized", index=False)
        final_df[1].to_excel(self.gold_path["hr_combined"]/"Summarized2.xlsx", sheet_name="Summarized2", index=False)
        final_df[2].to_excel(self.gold_path["hr_combined"]/"Summarized3.xlsx", sheet_name="Summarized3", index=False)

    def _temp_get_product(self, entry:str) -> str:
        """ 
            temporary function for aligning product classification with Traction for QBO accounts, will change for HP
        """
        entry = entry.lower()
        if "durum" in entry:
            return "Durum"
        elif "wheat" in entry:
            return "Wheat"
        elif "canola" in entry:
            return "Canola"
        elif ("chickpea" in entry) or ("garbanzo bean" in entry):
            return "Chickpeas"
        elif ("peas" in entry) or ("field pea" in entry):
            return "Peas"
        elif "barley" in entry:
            return "Barley"
        elif "green lentil" in entry:
            return "Green Lentils"
        elif "red lentil" in entry:
            return "Red Lentils"
        elif "oats" in entry:
            return "Oats"
        elif "corn" in entry:
            return "Corn"
        else:
            return "Others" 

    def _raw_inventory(self) -> None:
        """ 
            prepare the data from raw QBO table for inventory project: only extracting partial Invoice, SalesReceipt, and Journal Entry
        """
        print("\nStarting Inventory Project Transformation ...\n")
        corps = ["MFL", "MFUSA"]
        cols = ["TransactionDate", "TransactionType", "TransactionID", "Corp", "Qty", "AccID", "FarmID", "CustomerID",
                "DocNumber", "TransactionEntered", "Amount"]
        journal_cols = [col for col in cols if col != "Qty"]
        # read tables
        print("Loading raw tables ...")
        account = self.silver_acc.copy(deep=True)
        account = account[account["Corp"].isin(corps)]
        account = account[account["AccountType"] == "Income"]
        farm = pd.read_csv(self.silver_path["QBO"]["Dimension_time"]/"Farm.csv")
        farm = farm[farm["Corp"].isin(corps)]
        customer = pd.read_csv(self.silver_path["QBO"]["Dimension_time"]/"Customer.csv")
        customer = customer[customer["Corp"].isin(corps)]
        first_date = dt.datetime(2023,11,1)
        invoice = pd.read_csv(self.silver_path["QBO"]["Raw"]/"Invoice.csv")
        invoice = invoice[invoice["Corp"].isin(corps)]
        invoice["TransactionDate"] = pd.to_datetime(invoice["TransactionDate"])
        invoice = invoice[invoice["TransactionDate"]>=first_date]
        invoice = invoice[invoice["AccID"].isin(account.AccID.unique())]
        sales = pd.read_csv(self.silver_path["QBO"]["Raw"]/"SalesReceipt.csv")
        sales = sales[sales["Corp"].isin(corps)]
        sales["TransactionDate"] = pd.to_datetime(sales["TransactionDate"])
        sales = sales[sales["TransactionDate"]>=first_date]
        sales = sales[sales["AccID"].isin(account.AccID.unique())]
        journal = pd.read_csv(self.silver_path["QBO"]["Raw"]/"JournalEntry.csv",usecols=journal_cols)
        journal = journal[journal["AccID"].isin(account.AccID.unique())]
        journal["TransactionDate"] = pd.to_datetime(journal["TransactionDate"])
        journal = journal[journal["TransactionDate"]>=first_date]
        journal = journal[~journal["TransactionEntered"].str.contains("Delivered and not settled", na=False)]
        journal = journal[~journal["TransactionEntered"].str.contains("Grain Inventory Receivable Adjustment", na=False)]
        # combining tables
        print("Combining Fact Tables ...")
        invoice = invoice.loc[:,[col for col in cols if col in invoice.columns]]
        sales = sales.loc[:,[col for col in cols if col in sales.columns]]
        journal = journal.loc[:,[col for col in cols if col in journal.columns]]
        facts = pd.concat([invoice, sales, journal], ignore_index=True)
        del invoice, sales, journal
        # join facts with dimension tables
        facts = pd.merge(facts, account.loc[:,["AccID","AccNum","AccName","Category","Subcategory"]], on=["AccID"], how="left")
        facts = pd.merge(facts, farm.loc[:,["FarmID","FarmName"]], on=["FarmID"], how="left")
        facts = pd.merge(facts, customer.loc[:,["CustomerID","CustomerName"]], on=["CustomerID"], how="left")
        facts = facts[facts["Subcategory"]=="Grain - cash settlements"]
        print(f"Total Fact Entries - {len(facts)}")
        # product column
        facts["Product"] = facts["AccName"].apply(lambda x: self._temp_get_product(x))
        # saving file
        print("Saving Files ...")
        self.check_file(self.gold_path["inventory"])
        facts.to_excel(self.gold_path["inventory"]/"Excel"/"QBO_Grain_Settlements.xlsx", sheet_name="settlement", index=False)
        print("Finished\n")

    def _buget_process_input(self, inputdata_path:Path, processed_path:Path) -> None:
        """ 
            this function processes and saves budget totals for production, input (chem/fert/seed), produce budgets, and JD Lease
        """
        ## commodity prices - everything is CAD except Winter Wheat is converted to USD
        pricing = pd.read_csv(inputdata_path/"25-Grain-Pricing.csv")
        pricing.loc[pricing["Commodity"]=="WW", "ForecastPrice"] *= self.fx
        ## production budget
        budget_production = pd.read_csv(inputdata_path/"25-Grain-Revenue.csv")
        budget_production = budget_production.melt(
            id_vars=["Location", "Currency", "Type"],
            var_name="Commodity",
            value_name = "Amount"
        )
        budget_production = budget_production.fillna(value = {"Amount": 0})
        budget_production["Commodity"] = budget_production["Commodity"].replace({"Hay/Silage":"Hay"})
        budget_production.loc[((budget_production["Location"]=="Airdrie")&(budget_production["Commodity"]=="Hay")), "Commodity"] = "Silage" # only Airdrie has silage, others have hay
        budget_production_summary = pd.DataFrame(budget_production.groupby(["Location","Currency","Commodity"]).agg({"Amount": "prod"})).reset_index(drop=False)
        budget_production_summary = budget_production_summary.rename(columns={"Amount":"TotalYield"})
        ### merge yield with commodity price to calculate forecast production value of commodities
        budget_production_summary = pd.merge(budget_production_summary,pricing,on=["Commodity"], how="left")
        ### manual adjustments to prices
        budget_production_summary.loc[((budget_production_summary["Location"] == "Airdrie") & (budget_production_summary["Commodity"] == "Hay")), "ForecastPrice"] = 85
        budget_production_summary.loc[((budget_production_summary["Location"] == "Colorado (Genoa)") & (budget_production_summary["Commodity"] == "WW")), "ForecastPrice"] = 13.75
        budget_production_summary.loc[budget_production_summary["Location"] == "Yorkton", "ForecastPrice"] *= 2/3
        budget_production_summary["ForecastProductionCAD"] = budget_production_summary["TotalYield"] * budget_production_summary["ForecastPrice"]
        budget_production_summary = budget_production_summary[budget_production_summary["ForecastProductionCAD"].notna()]
        budget_production_summary = budget_production_summary[budget_production_summary["ForecastProductionCAD"]!=0]
        ### convert prices back to USD for a adjusted column
        budget_production_summary["ForecastProductionAdj"] = budget_production_summary.apply(lambda x: x["ForecastProductionCAD"] / self.fx if x["Currency"] == "USD" else x["ForecastProductionCAD"],axis=1)
        ### save production budget
        budget_production_summary.to_csv(processed_path/"budget_production.csv",index=False)
        ## input budget
        input_budget = pd.read_csv(inputdata_path/"25-Input-Budget.csv")
        input_budget = input_budget.drop(columns=["Total acres"])
        input_budget = input_budget.melt(
            id_vars = ["Location", "Type"],
            var_name = "Commodity",
            value_name = "Amount"
        )
        input_budget = input_budget.fillna(value = {"Amount": 0})
        input_budget.loc[((input_budget["Location"]=="Yorkton")&(input_budget["Type"].isin(["Fertilizer","Chemical","Seed"]))), "Amount"] *= 2/3
        input_budget.to_csv(processed_path/"input_budget.csv",index=False)
        ## labour budget
        labour_budget = pd.read_csv(inputdata_path/"25-Labour-Budget.csv")
        labour_budget = labour_budget.melt(
            id_vars = ["Location","Currency"],
            var_name = "Month",
            value_name = "LabourBudgetCAD"
        )
        labour_budget["LabourBudgetAdj"] = labour_budget.apply(lambda x: x["LabourBudgetCAD"]/self.fx if x["Currency"]=="USD" else x["LabourBudgetCAD"], axis=1)
        labour_budget.to_csv(processed_path/"labour_budget.csv",index=False)
        ## outlook budget
        outlook = pd.read_csv(inputdata_path/"25-Outlook-Detail.csv")
        outlook = outlook.melt(
            id_vars=["Type", "ProfitType"],
            var_name="Commodity",
            value_name="Amount"
        )
        outlook = outlook.fillna(value={"Amount": 0})
        outlook.to_csv(processed_path/"outlook_budget.csv", index=False)
        ## AZ budget
        az = pd.read_csv(inputdata_path / "25-AZ-Detail.csv")
        az = az.melt(
            id_vars=["Type", "ProfitType"],
            var_name="CommodityRaw",
            value_name="AmountCAD"
        )
        az = az.fillna(value={"AmountCAD": 0})
        az.to_csv(processed_path/"az_budget.csv", index=False)
        ## BC produce details
        bc = pd.read_csv(inputdata_path / "25-BC-Detail.csv")
        bc = bc.melt(
            id_vars=["Type", "ProfitType"],
            var_name="CommodityRaw",
            value_name="AmountCAD"
        )
        bc = bc.fillna(value={"AmountCAD": 0})
        bc.to_csv(processed_path/"bc_budget.csv", index=False)
        ## JD lease
        jdlease = pd.read_csv(inputdata_path/"25-JD-Lease-Summary.csv")
        jdlease = jdlease[jdlease["AllocatedCost25"] != 0]
        jdlease.to_csv(processed_path/"JD_lease.csv", index=False)

    def _budget_read_outsidedata(self,processed_path:Path) -> tuple[pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame]:
        """ 
            this function reads all the processed outside data and standardize the commodity and location naming
        """
        production_budget = pd.read_csv(processed_path/"budget_production.csv")
        input_budget = pd.read_csv(processed_path/"input_budget.csv")
        labour_budget = pd.read_csv(processed_path/"labour_budget.csv")
        outlook_budget = pd.read_csv(processed_path/"outlook_budget.csv")
        jdlease = pd.read_csv(processed_path/"JD_lease.csv")
        az_budget = pd.read_csv(processed_path/"az_budget.csv")
        bc_budget = pd.read_csv(processed_path/"bc_budget.csv")
        ## standardizing commodity naming
        production_rename_commodity = {"R Lentils":"Red Lentil", "G Lentils":"Green Lentil","Chickpeas":"Chickpea","Peas":"Field Pea", "WW": "Winter Wheat"}
        input_rename_commodity = {"R Lentils":"Red Lentil", "G Lentils":"Green Lentil","Chickpeas":"Chickpea", "WW": "Winter Wheat"}
        outlook_rename_commodity = {"Broccoli-cases/ac":"Broccoli", "Cabbage-lbs/ac":"Cabbage", "Carrots-lbs":"Carrot", "Cauliflower-cases/ac":"Cauliflower",
                                    "Table Potato-lbs":"Potato", "Seed Potato-lbs":"Potato", "Commercial Pumpkins-Bins/ac":"Pumpkin", "Strawberry Upick-lbs":"Strawberry",
                                    "Pumpkin Upick-pieces/ac":"Pumpkin", "Corn Maze-lbs":"Prairie Pathways", "WW": "Winter Wheat", "Corn (Sweet) Cobs":"Sweet Corn"}
        az_rename_commodity = {"Broccoli-cases/ac":"Broccoli", "Cabbage-lbs/ac":"Cabbage", "Pumpkins-Bins/ac":"Pumpkin", "WatermelonLG-bins/ac": "Watermelon",
                            "WatermelonMini-cases/ac": "Watermelon"}
        bc_rename_commodity = {"Broccoli-cases/ac":"Broccoli", "WatermelonLG-bins/ac": "Watermelon", "WatermelonMini-cases/ac": "Watermelon", "Pumpkins-Bins/ac":"Pumpkin",
                            "Squash-lbs": "Squash"}
        outlook_budget["CommodityRaw"] = outlook_budget["Commodity"]
        production_budget["Commodity"] = production_budget["Commodity"].replace(production_rename_commodity)
        input_budget["Commodity"] = input_budget["Commodity"].replace(input_rename_commodity)
        outlook_budget["Commodity"] = outlook_budget["Commodity"].replace(outlook_rename_commodity)
        az_budget["Commodity"] = az_budget["CommodityRaw"].replace(az_rename_commodity)
        bc_budget["Commodity"] = bc_budget["CommodityRaw"].replace(bc_rename_commodity)
        ## standardizing location naming - merge calderbank grain with Swift Current
        jdlease_rename_location = {"Swift Current Total":"Swift Current", "Regina Farm":"Regina", "Calderbank":"Swift Current",
                                "Airdrie":"Airdrie (grain)", "Eddystone":"Eddystone (grain)"}
        labour_rename_location = {"NexGen (avg met. ton)":"NexGen", "Cache/Fisher/Look":"Aschroft", "MF AZ":"Arizona (produce)", "Box Elder":"Havre", 
                                "BC Veg":"BritishColumbia (produce)","Monette Seeds CDN (avg met. ton)":"Monette Seeds", 
                                "BC Cattle (avg head)":"BritishColumbia (cattle)", "Eddystone Cattle (avg head)":"Eddystone (cattle)",
                                "Swift Current Cattle (avg head)":"Waldeck", "Aridrie Cattle (avg head)":"Airdrie (cattle)",
                                "Airdrie Farm":"Airdrie (grain)", "Eddystone Farm":"Eddystone (grain)","Calderbank":"Calderbank (cattle)"}
        input_rename_location =  {"Fly Creek/Camp 1":"Fly Creek", "Regina Farm":"Regina","Swift Current Total":"Swift Current", "Box Elder":"Havre", "Regina Farm":"Regina",
                                "Calderbank":"Calderbank (grain)","Airdrie":"Airdrie (grain)", "Eddystone":"Eddystone (grain)"}
        production_rename_location = {"Fly Creek/Camp 1":"Fly Creek", "Regina Farm":"Regina","Swift Current Total":"Swift Current", "Box Elder":"Havre", "Regina Farm":"Regina",
                                    "Colorado (Genoa)":"Colorado", "Calderbank":"Swift Current","Airdrie":"Airdrie (grain)", "Eddystone":"Eddystone (grain)"}
        input_budget["Location"] = input_budget["Location"].replace(input_rename_location)
        production_budget["Location"] = production_budget["Location"].replace(production_rename_location)
        labour_budget["Location"] = labour_budget["Location"].replace(labour_rename_location)
        jdlease["Location"] = jdlease["Location"].replace(jdlease_rename_location)
        ## put input budget (chem/fert/seed) into aggregated totals
        input_budget2 = input_budget.groupby(["Location","Type"]).agg({"Amount":"sum"}).reset_index(drop=False)
        ## aggregated totals for production budget and JD Lease
        production_budget = pd.DataFrame(production_budget.groupby(["Location","Currency","Commodity","ForecastPrice"]).agg({"TotalYield":"sum", "ForecastProductionCAD":"sum", "ForecastProductionAdj":"sum"}).reset_index(drop=False))
        jdlease = pd.DataFrame(jdlease.groupby(["Location","Country","Currency","TotalCost25"]).agg({"Acres25":"sum","AllocatedCost25":"sum"}).reset_index(drop=False))
        return input_budget2, production_budget, labour_budget, jdlease, az_budget, bc_budget, outlook_budget

    def _budget_process_produce(self, budget_rules:pd.DataFrame,budget:pd.DataFrame,sheetname:str) -> pd.DataFrame:
        """ 
            this function provides a standardized way to process produce budgets
        """
        budget_rules = budget_rules[budget_rules["SheetRef"] == sheetname].copy(deep=True)
        budget_rules["Commodity"] = budget_rules.apply(lambda x: self._identify_product(x,for_budget=True), axis=1)
        budget["Type"] = budget["Type"].str.strip()
        # gross income - by commodity
        reference = budget[budget["Type"].isin(["Acres","Unit Price","YieldPerAc"])]
        reference = reference.groupby(["Commodity","ProfitType","CommodityRaw"]).agg({"AmountCAD":"prod"}).reset_index(drop=False)
        reference = reference.groupby(["Commodity"]).agg({"AmountCAD":"sum"}).reset_index(drop=False)
        reference = reference.rename(columns={"AmountCAD":"TotalAmountCAD"})
        reference["Category"] = "Produce - production"
        if "outlook" in sheetname.lower():
            for item in ["Prairie Pathways", "Market Garden / CSA"]:
                reference.loc[reference["Commodity"] == item, "Category"] = "Produce - cash settlements"
        # seed expense - by commodity
        expense = budget[budget["Type"] == "Seed"].copy(deep=True)
        expense = expense.drop(columns="CommodityRaw")
        expense = expense.groupby(["Commodity"]).agg({"AmountCAD":"sum"}).reset_index(drop=False).rename(columns={"AmountCAD":"TotalAmountCAD"})
        expense["Category"] = "Seed"
        # other expense - Fertilizer/Chemical - not by commodity
        expense2 = budget[budget["Type"].isin(["Fertilizer","Chemical"])]
        expense2 = expense2.groupby(["Type"]).agg({"AmountCAD":"sum"}).reset_index(drop=False).rename(columns={"AmountCAD":"TotalAmountCAD"})
        expense2["Commodity"] = "Others"
        expense2 = expense2.rename(columns={"Type":"Category"})
        # combine
        budget_produce = pd.merge(budget_rules, pd.concat([reference,expense, expense2]), on=["Commodity","Category"], how="left")
        budget_produce = budget_produce.fillna(value={"TotalAmountCAD":0})
        budget_produce["AmountCAD"] = budget_produce.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)
        return budget_produce

    def _budget_get_transactions(self) -> pd.DataFrame:
        """ 
            get actuals
        """
        transactions = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
        # transactions = self.gold_pl.copy(deep=True)
        transactions = transactions[transactions["FiscalYear"] >= 2024]
        transactions["AccName"] = transactions["AccName"].str.strip()
        # transactions_location_rename = {"Calderbank":"Calderbank (cattle)"}
        # transactions["Location"] = transactions["Location"].replace(transactions_location_rename)
        return transactions

    def _create_budget(self, process_input:bool = False) -> None:
        """ 
            In Progress: this function generates budgets
        """
        print("\nCreating Budget\n")
        if not self.pl_exist:
            self._finance_operational()
        # self.gold_pl = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
        # self.fx = 1.3807
        inputdata_path = self.gold_path["budget"] / "Outside Data"
        processed_path = self.gold_path["budget"] / "Processed Data"
        rule_path = self.gold_path["budget"] / "Budget Rules"
        copied_path = self.gold_path["budget"]/"Copied Data"

        # load actuals
        transactions = self._budget_get_transactions()
        
        # process outside data
        if process_input:
            self._buget_process_input(inputdata_path=inputdata_path, processed_path=processed_path)
        
        # read outside data
        input_budget2, production_budget, labour_budget, jdlease, az_budget, bc_budget, outlook_budget = self._budget_read_outsidedata(processed_path=processed_path)

        # calculate Budgets
        ## outside data
        ### read rules
        budget_rules = pd.read_csv(rule_path/"OutsideData.csv")
        budget_rename_category = {"Seed - farm":"Seed"}
        budget_rules["Category"] = budget_rules["Category"].replace(budget_rename_category)
        ### separate locations into individual rows when they are separated with + in the rules df
        budget_rules["Location"] = budget_rules["Location"].str.split("+")
        budget_rules = budget_rules.explode("Location").reset_index(drop=True)
        ### extract formula
        budget_rules = budget_rules.melt(
            id_vars=["Location","Category","AccFull","SheetRef"],
            var_name="Month",
            value_name="Formula"
        )
        budget_rules = budget_rules.fillna(value={"Formula":"0"})
        budget_rules["Formula"] = budget_rules["Formula"].astype(str)
        budget_rules["Formula"] = budget_rules["Formula"].replace({"0": "*0"})
        ### calculating input budget for accounts per location
        budget_rules_input = budget_rules[budget_rules["SheetRef"] == "Input Budget"].copy(deep=True)
        #### workaround input budget for Airdrie grain 
        input_budget2.loc[((input_budget2["Location"]=="Airdrie (grain)")&(input_budget2["Type"]=="Acres")),"Type"] = "Custom work"
        ### merge budget rules with budget total per location
        budget_input = pd.merge(budget_rules_input,input_budget2.rename(columns={"Type":"Category","Amount":"TotalAmountCAD"}),on=["Location","Category"],how="left")
        #### revert back from workaround
        input_budget2.loc[((input_budget2["Location"]=="Airdrie (grain)")&(input_budget2["Type"]=="Custom work")),"Type"] = "Acres"
        ### apply the formula to compute per month
        budget_input["AmountCAD"] = budget_input.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"), axis=1)

        ## production budget
        ### combine Hay and Silage
        production_budget["Commodity"] = production_budget["Commodity"].replace({"Hay":"Hay/Silage", "Silage":"Hay/Silage"})
        ### add commodity column to budget rules
        budget_rules_production = budget_rules[budget_rules["SheetRef"] == "Production Budget"].copy(deep=True)
        budget_rules_production["Commodity"] = budget_rules_production.apply(lambda x: self._identify_product(x, for_budget=True), axis=1)
        ### merge budget rules with budget totals
        budget_production = pd.merge(budget_rules_production,production_budget.loc[:,["Location","Commodity","ForecastProductionCAD"]].rename(columns={"ForecastProductionCAD":"TotalAmountCAD"}),
                                         on = ["Location", "Commodity"], how="left")
        budget_production = budget_production.fillna(value={"TotalAmountCAD":0})
        ### compute budget
        budget_production["AmountCAD"] = budget_production.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)

        ## labour budget
        budget_rules_labour = budget_rules[budget_rules["SheetRef"] == "Labour Budget"].copy(deep=True)
        budget_labour = pd.merge(budget_rules_labour, labour_budget.loc[:,["Location","Month","LabourBudgetCAD"]].rename(columns={"LabourBudgetCAD":"AmountCAD"}),
                                    on=["Location","Month"],how="left")
        
        ## produce budgets
        ### BC
        budget_bc_produce = self._budget_process_produce(budget_rules=budget_rules,budget=bc_budget,sheetname="BC Produce Details")
        ### AZ
        budget_az_produce = self._budget_process_produce(budget_rules=budget_rules,budget=az_budget,sheetname="AZ Details")
        ### outlook
        budget_outlook = self._budget_process_produce(budget_rules=budget_rules,budget=outlook_budget.rename(columns={"Amount":"AmountCAD"}),sheetname="Outlook Details")

        ## JD lease
        budget_rules_jd = budget_rules[budget_rules["SheetRef"]=="JD Lease"].copy(deep=True)
        budget_equipment = pd.merge(budget_rules_jd, jdlease.loc[:,["Location","AllocatedCost25"]].rename(columns={"AllocatedCost25":"TotalAmountCAD"}),
                                        on = "Location", how = "left")
        budget_equipment = budget_equipment.fillna(value={"TotalAmountCAD":0})
        budget_equipment["AmountCAD"] = budget_equipment.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)

        ## adjustment for Swift Current
        months = ["April", "July"]
        for month in months:
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Fertilizer")&(budget_input["Month"]==month)),"TotalAmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Fertilizer")&(budget_input["Month"]==month)),"TotalAmountCAD"].item()
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Fertilizer")&(budget_input["Month"]==month)),"AmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Fertilizer")&(budget_input["Month"]==month)),"AmountCAD"].item()
        months = ["June", "September"]
        for month in months:
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Chemical")&(budget_input["Month"]==month)),"TotalAmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Chemical")&(budget_input["Month"]==month)),"TotalAmountCAD"].item()
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Chemical")&(budget_input["Month"]==month)),"AmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Chemical")&(budget_input["Month"]==month)),"AmountCAD"].item()
        months = ["May", "June", "September"]
        for month in months:
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Seed")&(budget_input["Month"]==month)),"TotalAmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Seed")&(budget_input["Month"]==month)),"TotalAmountCAD"].item()
            budget_input.loc[((budget_input["Location"]=="Swift Current")&(budget_input["Category"]=="Seed")&(budget_input["Month"]==month)),"AmountCAD"] += \
            budget_input.loc[((budget_input["Location"]=="Calderbank (grain)")&(budget_input["Category"]=="Seed")&(budget_input["Month"]==month)),"AmountCAD"].item()
        
        # arithmetic rules
        arithmetic = pd.read_csv(rule_path/"Arithmetic.csv")
        ## faltten location
        arithmetic["Location"] = arithmetic["Location"].str.split("+")
        arithmetic = arithmetic.explode("Location").reset_index(drop=True)
        arithmetic_rules = arithmetic.melt(
            id_vars=["Location","Category","AccFull", "AccRef", "FixedRef"],
            var_name="Month",
            value_name="FormulaFull"
        )
        ## housekeeping
        arithmetic_rules = arithmetic_rules.fillna(value={"FormulaFull":"FY-1*0"})
        arithmetic_rules["FormulaFull"] = arithmetic_rules["FormulaFull"].astype(str)
        arithmetic_rules["FormulaFull"] = arithmetic_rules["FormulaFull"].replace({"0":"FY-1*0"})
        arithmetic_rules["ReferenceYear"] = arithmetic_rules["FormulaFull"].str.slice(0,4)
        arithmetic_rules["Formula"] = arithmetic_rules["FormulaFull"].str.slice(4)
        arithmetic_rules = arithmetic_rules.fillna(value={"Formula": "0"})
        arithmetic_rules["Formula"] = arithmetic_rules["Formula"].astype(str)
        arithmetic_rules["Formula"] = arithmetic_rules["Formula"].replace({"0":"*0"})
        ## separating Fixed records
        arithmetic_rules_fixed = arithmetic_rules[arithmetic_rules["AccRef"] == "Fixed"].copy(deep=True)
        arithmetic_rules = arithmetic_rules[arithmetic_rules["AccRef"]!="Fixed"].copy(deep=True)

        ## process fixed records
        arithmetic_rules_fixed = arithmetic_rules_fixed.drop(columns=["FormulaFull","ReferenceYear"]).rename(columns={"FixedRef":"TotalAmountCAD"})
        arithmetic_rules_fixed["AmountCAD"] = arithmetic_rules_fixed.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)

        ## Extract Account Info
        arithmetic_rules["AccNum"] = arithmetic_rules["AccRef"].apply(lambda x: "".join(x.split(" ")[0:2]))
        arithmetic_rules["AccName"] = arithmetic_rules["AccRef"].apply(lambda x: (" ".join(x.split(" ")[2:]).strip()))
        assert "Fixd" not in arithmetic_rules.ReferenceYear.unique(), "Fixd records incorrectly classified"
        ## separate FY-1 & FY+1
        arithmetic_rules_prior = arithmetic_rules[arithmetic_rules["ReferenceYear"] == "FY-1"].copy(deep=True)
        arithmetic_rules = arithmetic_rules[arithmetic_rules["ReferenceYear"] == "FY+1"].copy(deep=True)

        ## process FY-1 with actuals
        actuals = transactions.groupby(["Location", "AccNum", "AccName", "FiscalYear"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
        arithmetic_rules_prior["FiscalYear"] = self.currentFY - 1
        assert len(actuals[actuals.duplicated(subset=["AccNum","FiscalYear","Location"],keep=False)]) == 0, "Duplicated AccNum detected for FY-1 Actuals"
        budget_prior = pd.merge(arithmetic_rules_prior,actuals.rename(columns={"AmountDisplay":"TotalAmountCAD"}),
                                on = ["Location","AccNum","FiscalYear"], how="left")
        budget_prior = budget_prior.fillna(value={"TotalAmountCAD": 0})
        budget_prior["AmountCAD"] = budget_prior.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"), axis=1)

        ## processing FY+1 with current budget
        ### budget sales that is based on production budget input sheet
        arithmetic_rules_sales = arithmetic_rules[arithmetic_rules["Category"].str.contains("cash settlements")].copy(deep=True)
        production_reference = pd.concat([budget_production.copy(deep=True), budget_outlook.copy(deep=True), budget_az_produce.copy(deep=True),budget_bc_produce.copy(deep=True)])
        production_reference = production_reference.groupby(["Location","AccFull"]).agg({"AmountCAD":"sum"}).reset_index(drop=False)
        budget_sales = pd.merge(arithmetic_rules_sales,production_reference.rename(columns={"AccFull":"AccRef"}), on=["Location","AccRef"], how="left")
        budget_sales = budget_sales.rename(columns={"AmountCAD":"TotalAmountCAD"})
        budget_sales["AmountCAD"] = budget_sales.apply(lambda x: eval(f"{x["TotalAmountCAD"]}{x["Formula"]}"),axis=1)
        budget_prior = pd.concat([budget_prior, budget_sales],ignore_index=True)

        ### budget inventory adjustment 
        arithmetic_rules_inventory = arithmetic_rules[arithmetic_rules["Category"].str.contains("inventory adjustment",case=False)].copy(deep=True)
        budget_inventory = pd.merge(arithmetic_rules_inventory, budget_prior.loc[:,["Location","AccFull","Month","AmountCAD"]].rename(columns={"AccFull":"AccRef"}),
                                on = ["Location","AccRef", "Month"], how = "left")
        budget_inventory["AmountCAD"] = -budget_inventory["AmountCAD"]
        budget_prior = pd.concat([budget_prior, budget_inventory], ignore_index=True)

        ## combine with fixed budgets
        budget_prior = pd.concat([budget_prior,arithmetic_rules_fixed],ignore_index=True)

        # copied data
        budget_copy = pd.read_csv(copied_path/"Copied Data.csv")
        budget_copy = budget_copy.melt(
            id_vars=["Location","Category","AccFull"],
            var_name = "Month",
            value_name = "AmountCAD"
        )
        budget_copy = budget_copy.fillna(value={"AmountCAD":0})
        budget_copy["AmountCAD"] = budget_copy["AmountCAD"].astype(float)
        budget_copy["FiscalYear"] = self.currentFY
        budget_copy["AccRef"] = "Copy"
        budget_copy["ReferenceYear"] = "NA"
        budget_copy["Formula"] = "NA"
        budget_copy["TotalAmountCAD"] = budget_copy["AmountCAD"]
        budget_copy.loc[budget_copy["Location"]=="Seeds USA", "AmountCAD"] *= self.fx

        # combining all budgets
        budget_outside = pd.concat([budget_input,budget_production,budget_labour,budget_equipment, budget_outlook, budget_az_produce, budget_bc_produce],ignore_index=True)
        budget_outside = budget_outside.drop(columns=["Commodity"])
        budget_prior = budget_prior.drop(columns=["FormulaFull","AccNum","AccName_x", "AccName_y", "AccName", "FixedRef"])
        budget_all = pd.concat([budget_outside,budget_prior,budget_copy],ignore_index=True)
        budget_all["AccNum"] = budget_all["AccFull"].apply(lambda x: "".join(x.split(" ")[0:2]))
        budget_all["AccName"] = budget_all["AccFull"].apply(lambda x: " ".join(x.split(" ")[2:]))
        budget_all["FiscalYear"] = self.currentFY 
        budget_all["DataType"] = "Budget"
        budget_all.loc[budget_all["Category"].str.contains("inventory adjustment",case=False), "AmountCAD"] *= -1 # turn the sign positive for inventory adjustments

        # save
        self.check_file(self.gold_path["budget"]/"OutputFile")
        budget_all.to_csv(self.gold_path["budget"]/"OutputFile"/"budget_all.csv", index=False)

    def _budget_update(self, force_create:bool=False, force_process_input:bool=False) -> None:
        """ 
            generate/update the actuals from the budget system
        """
        print("\nGenerating/Updating Actuals for budget system\n")
        # if not self.pl_exist:
        #     self._finance_operational()
        self.operation_acc = pd.read_csv(self.gold_path["finance_operational"]/"Account_table_budget.csv")
        self.gold_pl = pd.read_csv(self.gold_path["finance_operational"]/"PL.csv")
        self.fx = 1.3807
        budget_path = self.gold_path["budget"]/"OutputFile"/"budget_all.csv"
        if not Path.exists(budget_path) or force_create:
            self._create_budget(process_input=force_process_input)
        budget = pd.read_csv(budget_path)
        budget = budget.loc[:,["Location", "SheetRef", "Month", "Formula", "TotalAmountCAD", "AmountCAD", "AccRef", "ReferenceYear","FiscalYear", "AccNum", "DataType", "Category"]]
        budget_location_rename = {"Airdrie (grain)": "Airdrie", "Airdrie (cattle)": "Airdrie", "Calderbank (cattle)": "Calderbank",
                                  "Airdrie (corporate)": "Airdrie", "Seeds USA":"Arizona (produce)"}
        budget["Location"] = budget["Location"].replace(budget_location_rename)
        category_mapping = budget.loc[:,["AccNum", "Category"]].drop_duplicates()
        # organize Actuals
        transactions = self._budget_get_transactions()
        actuals_all = transactions.groupby(["Location","AccNum", "FiscalYear", "Month"]).agg({"AmountDisplay":"sum"}).reset_index(drop=False)
        actuals_all = actuals_all[actuals_all["FiscalYear"] == self.currentFY]
        actuals_all["DataType"] = "Actual"
        actuals_all = actuals_all.rename(columns={"AmountDisplay": "AmountCAD"})
        actuals_all = pd.merge(actuals_all,category_mapping,on="AccNum",how="left")
        actuals_all.to_csv(self.gold_path["budget"]/"OutputFile"/"actuals_all.csv", index=False)
        print(f"Location Unaccounted for in budget: {(set(budget.Location.unique()) - set(actuals_all.Location.unique()))}")
        # combine everything
        all_all = pd.concat([budget,actuals_all],ignore_index=True)
        all_all["FXRate"] = self.fx
        # reroute accounts for changing acc numbers
        accnum_reroute = {"MFL405101": "MFL405110", "MFL405102":"MFL405120", "MFL405103":"MFL405130",
                            "MSL585000":"MSL562505"}
        all_all["AccNum"] = all_all["AccNum"].replace(to_replace=accnum_reroute)
        # operational classification
        assert len(self.operation_acc[self.operation_acc.duplicated(subset=["AccNum"],keep=False)]) == 0, "Duplicated AccNum Detected - Operational Accounts Classification"
        all_all = pd.merge(all_all, self.operation_acc.loc[:,["AccNum", "AccID"]], on="AccNum", how="left")
        print(f"Total amount unaccounted for because of accnum mismatching - ${all_all[all_all["AccID"].isna()].AmountCAD.sum():,}")
        # classify pillars
        all_all["Pillar"] = all_all.apply(lambda x: self._pillar_classification(x), axis=1)
        # save
        self.check_file(self.gold_path["budget"]/"OutputPowerBI")
        all_all.to_excel(self.gold_path["budget"]/"OutputPowerBI"/"BudgetActual.xlsx", sheet_name="Budget", index=False)

    def run(self, force_run_time:bool=False, force_create_budget:bool=False, force_process_budget_input:bool=False) -> None:
        start = perf_counter()

        self._weekly_banking()
        self._finance_operational()
        self._payroll_project()
        self._budget_update(force_create=force_create_budget, force_process_input=force_process_budget_input)
        if force_run_time or (self.today.weekday() in [0, 2, 6]): self._QBOTime_project()
        self._hr_summary()
        self._raw_inventory()

        end = perf_counter()
        print(f"\nProjects Transformation Finished with {(end-start)/60:.3f} minutes\n")



In [152]:
self = Projects()
# self.run(force_run_time=False)

In [153]:
self._finance_operational()


Starting Finance Operational Project Transformation

location unaccounted for - [nan]
Revising Signs ...
Saving ...


In [154]:
self._budget_update()


Generating/Updating Actuals for budget system

Location Unaccounted for in budget: {'Calderbank (grain)'}
Total amount unaccounted for because of accnum mismatching - $0.0


In [146]:
a

,Location,SheetRef,Month,Formula,TotalAmountCAD,AmountCAD,AccRef,ReferenceYear,FiscalYear,AccNum,DataType,Category,FXRate,AccID
0,Regina,Input Budget,November,/4,4154073.0,1038518.25,NaN,NaN,2025,MFL501000,Budget,Fertilizer,1.3807,MFL270
1,Raymore,Input Budget,November,/4,1460318.0,365079.50,NaN,NaN,2025,MFL501000,Budget,Fertilizer,1.3807,MFL270
2,Prince Albert,Input Budget,November,/2,2496507.0,1248253.50,NaN,NaN,2025,MFL501000,Budget,Fertilizer,1.3807,MFL270
3,The Pas,Input Budget,November,*0,3008469.0,0.00,NaN,NaN,2025,MFL501000,Budget,Fertilizer,1.3807,MFL270
4,Hafford,Input Budget,November,*0.47,4213666.0,1980423.02,NaN,NaN,2025,MFL501000,Budget,Fertilizer,1.3807,MFL270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60643,Yorkton,NaN,June,NaN,NaN,46156.06,NaN,NaN,2025,MFL572000,Actual,Fuel and lubrication,1.3807,MFL305
60644,Yorkton,NaN,February,NaN,NaN,77686.40,NaN,NaN,2025,MFL574000,Actual,Custom work,1.3807,MFL310
60645,Yorkton,NaN,February,NaN,NaN,545.94,NaN,NaN,2025,MFL584600,Actual,Telephone and utilities,1.3807,MFL341
60646,Yorkton,NaN,May,NaN,NaN,535.72,NaN,NaN,2025,MFL584600,Actual,Telephone and utilities,1.3807,MFL341


In [147]:
b = a[a["AccID"].isna()]
b.groupby(["AccNum", "DataType"]).agg({"AmountCAD":"sum"})

,,AmountCAD
AccNum,DataType,
MFBC405400,Budget,0.0
MFBC531010,Budget,0.0
MFBC536140,Budget,0.0
MFBC539500,Budget,0.0
MFBC634020,Budget,0.0
MFBCMinistry,Budget,0.0
MFL535008,Budget,0.0


In [132]:
b[b["AccNum"] == "MFBC629000"]

,Location,SheetRef,Month,Formula,TotalAmountCAD,AmountCAD,AccRef,ReferenceYear,FiscalYear,AccNum,DataType,Category,FXRate,AccID
53981,BritishColumbia (produce),NaN,July,NaN,NaN,138.38,NaN,NaN,2025,MFBC629000,Actual,NaN,1.3807,NaN


In [80]:
print(len(a))
a = a[a["Active"]]
len(a)

3422


3168

In [82]:
b = a[a.duplicated(subset=["AccNum"],keep=False)]
b[b["AccNum"].notna()]

,AccID,AccNum,AccName,FullyQualifiedName,Classification,AccountType,SubAccount,Active,CurrencyID,ParentAccID,...,DetailType,AccountingType,Profitem,Category,Subcategory,DisplayName,OperationProfType,OperationCategory,OperationSubCategory,Commodity


# budget seasonality

## isolate last year's transactions

In [14]:
transactions = self._budget_get_transactions()
transactions["TransactionDate"] = pd.to_datetime(transactions["TransactionDate"])
transactions.FiscalYear.value_counts()

FiscalYear
2024    52248
2025    44209
Name: count, dtype: int64

In [13]:
self.currentFY

2025

In [16]:
transactions1 = transactions[transactions["FiscalYear"]==self.currentFY-1]
transactions1.FiscalYear.value_counts()

FiscalYear
2024    52248
Name: count, dtype: int64

## summarize for each account and amount of transactions per month

In [18]:
transactions1.columns

Index(['TransactionDate', 'TransactionType', 'TransactionID_partial',
       'DocNumber', 'Name', 'NameID', 'LocationRaw', 'FarmID', 'Class', 'Memo',
       'SplitAcc', 'SplitAccID', 'Amount', 'Balance', 'AccName', 'AccID',
       'Corp', 'AccNum', 'ClassID', 'AmountAdj', 'AmountCAD', 'FXRate',
       'Country', 'FiscalYear', 'Month', 'Location', 'Pillar',
       'AmountDisplay'],
      dtype='object')

# reconstruct account classification

In [19]:
import yaml

In [28]:
with open("acc_classification.yaml", "r", encoding="utf-8") as f:
    data = yaml.safe_load(f)
data

{'Core Operation': {'Production': {'Grain Production': ['MFAZ155',
    'MFAZ156',
    'MFAZ157',
    'MFAZ159',
    'MFAZ160',
    'MFL718',
    'MFL720',
    'MFL721',
    'MFL722',
    'MFL723',
    'MFL724',
    'MFL725',
    'MFL726',
    'MFL728',
    'MFL802',
    'MFUSA247',
    'MFUSA250',
    'MFUSA251',
    'MFUSA252',
    'MFUSA253',
    'MFUSA256',
    'MFUSA257'],
   'Produce Production': ['MFAZ170',
    'MFAZ179',
    'MFAZ180',
    'MFAZ181',
    'MFAZ232',
    'MFBC517',
    'MFBC518',
    'MFBC519',
    'MFBC520',
    'MFBC617',
    'MFBC645',
    'MFL730',
    'MFL731',
    'MFL732',
    'MFL733',
    'MFL734',
    'MFL735',
    'MFL736',
    'MFL746'],
   'Cattle Production': ['MFBC444', 'MFL670']},
  'Expense': {'Seed Expense': ['MFAZ1150040006',
    'MFAZ186',
    'MFAZ187',
    'MFAZ189',
    'MFAZ191',
    'MFAZ211',
    'MFAZ212',
    'MFAZ234',
    'MFAZ238',
    'MFAZ95',
    'MFBC453',
    'MFBC523',
    'MFBC524',
    'MFBC526',
    'MFBC531',
    'MFBC532',

In [29]:
rows = [(l1, l2, l3, v) 
        for l1, l1_inner in data.items() 
        for l2, l2_inner in l1_inner.items() 
        for l3, l3_inner in l2_inner.items() 
        for v in l3_inner]
df = pd.DataFrame(rows, columns=["col1", "col2", "col3", "acc"])
df

,col1,col2,col3,acc
0,Core Operation,Production,Grain Production,MFAZ155
1,Core Operation,Production,Grain Production,MFAZ156
2,Core Operation,Production,Grain Production,MFAZ157
3,Core Operation,Production,Grain Production,MFAZ159
4,Core Operation,Production,Grain Production,MFAZ160
...,...,...,...,...
1085,Accounting,Reconciliation Discrepancies,NA,MFBC407
1086,Accounting,Reconciliation Discrepancies,NA,MFL376
1087,Accounting,Reconciliation Discrepancies,NA,MFL784
1088,Accounting,Reconciliation Discrepancies,NA,MFUSA1150040002
